In [ ]:
import os
import pandas as pd
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.metrics import mean_squared_error
import gc

# Set device for PyTorch
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Function to free up memory
def clear_memory():
    torch.cuda.empty_cache()
    gc.collect()

# Load Data
anime = pd.read_csv("anime.csv")
ratings = pd.read_csv("rating.csv")

# Preprocessing Anime Data
anime.genre.fillna("NULL", inplace=True)
anime.type.fillna("NULL", inplace=True)
anime.rating.fillna(anime.rating.mean(), inplace=True)

# Filter out bad anime IDs from the ratings
valid_anime_ids = set(anime.anime_id)
ratings = ratings[ratings.anime_id.isin(valid_anime_ids)]
ratings = ratings[ratings.rating != -1]  # Exclude unrated items (-1)

# Feature Engineering on Anime Data
anime['genre'] = anime['genre'].apply(lambda x: " ".join(x.split(" ")).split(", "))
genres = list(set(g for sublist in anime['genre'] for g in sublist))
for genre in genres:
    anime['genre_' + genre] = anime['genre'].apply(lambda x: 1 if genre in x else 0)

# One-hot encoding for anime type
anime = pd.concat([anime, pd.get_dummies(anime['type'], prefix='type')], axis=1)
anime.drop(['genre', 'type', 'episodes', 'name'], axis=1, inplace=True)
anime.rename(columns={"rating": "avgRating"}, inplace=True)

# Merging Ratings with Anime Data
ratings = pd.merge(ratings, anime, on="anime_id", how="left")
ratings.fillna(0, inplace=True)

# Preparing Training and Validation Sets
train = ratings.sample(frac=0.8, random_state=42)
validation = ratings.drop(train.index)

# Normalize Numerical Features
scalers = {
    'avgRating': MinMaxScaler(),
    'members': MinMaxScaler()
}

for feature, scaler in scalers.items():
    train[feature] = scaler.fit_transform(train[[feature]])
    validation[feature] = scaler.transform(validation[[feature]])

# Encode Anime IDs
anime_encoder = LabelEncoder()
train['anime_id'] = anime_encoder.fit_transform(train['anime_id'])
validation['anime_id'] = anime_encoder.transform(validation['anime_id'])

# Model Definition
class RecommendationNet(nn.Module):
    def __init__(self, num_users, num_animes, num_features):
        super(RecommendationNet, self).__init__()
        self.user_embedding = nn.Embedding(num_users, 100)
        self.anime_embedding = nn.Embedding(num_animes, 100)
        self.fc1 = nn.Linear(200 + num_features, 128)
        self.fc2 = nn.Linear(128, 32)
        self.fc3 = nn.Linear(32, 1)
        self.dropout = nn.Dropout(0.2)
    
    def forward(self, x):
        user = x[:, 0].long()
        anime = x[:, 1].long()
        other_features = x[:, 2:]
        
        user_vec = self.user_embedding(user)
        anime_vec = self.anime_embedding(anime)
        x = torch.cat((user_vec, anime_vec, other_features), dim=1)
        
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))
        return x

# Initialize Model, Optimizer, and Loss
num_users = ratings['user_id'].nunique()
num_animes = len(anime_encoder.classes_)
num_features = train.shape[1] - 3  # Exclude 'user_id', 'anime_id', 'rating'

model = RecommendationNet(num_users, num_animes, num_features).to(device)
optimizer = optim.Adagrad(model.parameters(), lr=0.001)

# Prepare DataLoader
def prepare_data_loader(data, batch_size=32):
    tensor_data = torch.tensor(data.to_numpy(), dtype=torch.float32)
    dataset = TensorDataset(tensor_data)
    return DataLoader(dataset, batch_size=batch_size, shuffle=True)

train_loader = prepare_data_loader(train, batch_size=32)
val_loader = prepare_data_loader(validation, batch_size=32)

# Training Loop with Gradient Accumulation
def train_model(model, train_loader, val_loader, optimizer, num_epochs=10, accumulation_steps=4):
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0
        
        for i, (batch,) in enumerate(train_loader):
            batch = batch.to(device)
            X = batch[:, 1:]
            y = batch[:, :1]
            
            optimizer.zero_grad()
            predictions = model(X)
            loss = F.mse_loss(predictions, y)
            loss = loss / accumulation_steps  # Scale the loss
            loss.backward()

            # Gradient accumulation
            if (i + 1) % accumulation_steps == 0:
                optimizer.step()
                clear_memory()  # Free memory after each accumulation step
                
            train_loss += loss.item()
        
        # Validation
        model.eval()
        with torch.no_grad():
            val_loss = 0
            for val_batch, in val_loader:
                val_batch = val_batch.to(device)
                val_X = val_batch[:, 1:]
                val_y = val_batch[:, :1]
                val_predictions = model(val_X)
                val_loss += F.mse_loss(val_predictions, val_y).item()
            
            val_loss = np.sqrt(val_loss / len(val_loader))
        
        print(f"Epoch {epoch+1}/{num_epochs}, Training Loss: {train_loss/len(train_loader):.4f}, Validation Error: {val_loss:.4f}")
        clear_memory()

# Train the model
train_model(model, train_loader, val_loader, optimizer)

# Recommendations
def recommend_animes(model, user_id, top_n=10):
    user_data = production_data[production_data.user_id == user_id].copy()
    user_data['anime_id'] = anime_encoder.transform(user_data['anime_id'])
    user_tensor = torch.tensor(user_data.to_numpy(), dtype=torch.float32).to(device)
    user_tensor[:, 0] = model(user_tensor[:, 1:]).cpu().detach().numpy().reshape(-1)
    recommendations = user_data.iloc[user_tensor[:, 0].argsort()[-top_n:][::-1]]
    return anime[anime.anime_id.isin(recommendations['anime_id'].tolist())]

# Get recommendations for user 1
recommended_animes = recommend_animes(model, user_id=1)
print(recommended_animes)
